In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
!pip install keras_tuner
import keras_tuner as kt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images = training_images.reshape(60000, 28, 28, 1)
training_images = training_images / 255.0
test_images = test_images.reshape(10000, 28, 28, 1)
test_images = test_images/255.0

untuned_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

untuned_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

untuned_model.fit(training_images, training_labels, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.4357 - accuracy: 0.8413
Epoch 2/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2928 - accuracy: 0.8919
Epoch 3/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2479 - accuracy: 0.9084
Epoch 4/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2160 - accuracy: 0.9192
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1882 - accuracy: 0.9293
Epoch 6/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1670 - accuracy: 0.9369
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1442 - accuracy: 0.9440
Epoch 8/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1277 - accuracy: 0.9513
Epoch 9/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1118 - accuracy: 0.9577
Epoch 10/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.097

In [4]:
yp=untuned_model.predict(test_images)
y_pred=[]
for i in range(yp.shape[0]):
  y_pred.append(np.argmax(yp[i]))

from sklearn.metrics import classification_report

print(classification_report(test_labels,y_pred))

313/313 [==============================] - 1s 2ms/step
              precision    recall  f1-score   support

           0       0.84      0.87      0.86      1000
           1       1.00      0.97      0.99      1000
           2       0.84      0.89      0.86      1000
           3       0.93      0.91      0.92      1000
           4       0.88      0.85      0.86      1000
           5       0.98      0.98      0.98      1000
           6       0.76      0.73      0.74      1000
           7       0.95      0.98      0.97      1000
           8       0.97      0.99      0.98      1000
           9       0.98      0.96      0.97      1000

    accuracy                           0.91     10000
   macro avg       0.91      0.91      0.91     10000
weighted avg       0.91      0.91      0.91     10000



# Trying Hyperparameter tuning

In [5]:
from keras_tuner import RandomSearch
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten

In [8]:
def build_model(hp):
  counter=0
  model=Sequential([
       keras.layers.Conv2D(
           filters=hp.Int('conv1_filter', min_value=32, max_value=128, step=16),
           kernel_size=hp.Choice('conv1_kernel', values=[3,5]),
           activation=hp.Choice('conv1_activation', values=['relu', 'tanh', 'sigmoid','softmax']),
           input_shape=(28,28,1)
       ),
       keras.layers.MaxPooling2D(2, 2),
       keras.layers.Conv2D(
           filters=hp.Int('conv2_filter', min_value=32, max_value=64, step=16),
           kernel_size=hp.Choice('conv2_kernel', values=[3,5]),
           activation=hp.Choice('conv2_activation', values=['relu', 'tanh', 'sigmoid','softmax']),
       ),
       keras.layers.MaxPooling2D(2, 2),
       keras.layers.Flatten(),

       keras.layers.Dense(units=hp.Int('units',min_value=8,max_value=256,step=16),
              activation=hp.Choice('Activation',values=['relu','tanh','sigmoid'])
              ),      

       keras.layers.Dense(10, activation='softmax')      
   ])
  optimizer=hp.Choice("optimizer",values=['adam','rmsprop','adadelta','sgd','nadam'])
  model.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy',metrics=['accuracy'])   

  return model

In [9]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=10,directory='tuning',project_name='version1')

In [10]:
tuner.search(training_images, training_labels, epochs=10,validation_data=(test_images, test_labels))

Trial 10 Complete [00h 01m 10s]
val_accuracy: 0.8443999886512756

Best val_accuracy So Far: 0.8903999924659729
Total elapsed time: 00h 14m 14s


In [11]:
tuner.results_summary()

Results summary
Results in tuning/version1
Showing 10 best trials
Trial summary
Hyperparameters:
conv1_filter: 80
conv1_kernel: 5
conv1_activation: sigmoid
conv2_filter: 64
conv2_kernel: 3
conv2_activation: tanh
units: 152
Activation: tanh
optimizer: rmsprop
Score: 0.8903999924659729
Trial summary
Hyperparameters:
conv1_filter: 48
conv1_kernel: 3
conv1_activation: sigmoid
conv2_filter: 48
conv2_kernel: 5
conv2_activation: sigmoid
units: 104
Activation: relu
optimizer: nadam
Score: 0.8853999972343445
Trial summary
Hyperparameters:
conv1_filter: 96
conv1_kernel: 3
conv1_activation: sigmoid
conv2_filter: 32
conv2_kernel: 3
conv2_activation: softmax
units: 104
Activation: relu
optimizer: adam
Score: 0.868399977684021
Trial summary
Hyperparameters:
conv1_filter: 80
conv1_kernel: 3
conv1_activation: sigmoid
conv2_filter: 48
conv2_kernel: 3
conv2_activation: softmax
units: 8
Activation: tanh
optimizer: adam
Score: 0.8682000041007996
Trial summary
Hyperparameters:
conv1_filter: 48
conv1_kernel

In [15]:
tuned_model=tuner.get_best_models(num_models=1)[0]

In [16]:
tuned_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 80)        2080      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 80)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 10, 64)        46144     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dense (Dense)               (None, 152)               2

In [17]:
tuned_model.fit(training_images, training_labels, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2544 - accuracy: 0.9068
Epoch 2/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2465 - accuracy: 0.9106
Epoch 3/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2393 - accuracy: 0.9122
Epoch 4/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2350 - accuracy: 0.9144
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2272 - accuracy: 0.9167
Epoch 6/10
1875/1875 [==============================] - 7s 3ms/step - loss: 0.2214 - accuracy: 0.9196
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2157 - accuracy: 0.9220
Epoch 8/10
1875/1875 [==============================] - 7s 3ms/step - loss: 0.2123 - accuracy: 0.9241
Epoch 9/10
1875/1875 [==============================] - 7s 3ms/step - loss: 0.2072 - accuracy: 0.9251
Epoch 10/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.198

In [18]:
yp=tuned_model.predict(test_images)
y_pred=[]
for i in range(yp.shape[0]):
  y_pred.append(np.argmax(yp[i]))

print(classification_report(test_labels,y_pred))

313/313 [==============================] - 1s 2ms/step
              precision    recall  f1-score   support

           0       0.84      0.87      0.86      1000
           1       1.00      0.98      0.99      1000
           2       0.80      0.89      0.85      1000
           3       0.90      0.92      0.91      1000
           4       0.86      0.81      0.83      1000
           5       0.99      0.97      0.98      1000
           6       0.77      0.69      0.73      1000
           7       0.94      0.98      0.96      1000
           8       0.98      0.98      0.98      1000
           9       0.97      0.96      0.96      1000

    accuracy                           0.90     10000
   macro avg       0.90      0.90      0.90     10000
weighted avg       0.90      0.90      0.90     10000

